In [1]:
#Definindo finais de semana
mes = "06"
lista_fds = ['01','02','08','09','15','16','22','23','29','30']

In [2]:
####################################-BIBLIOTECAS-####################################

#manipulação de dados
import pandas as pd
import numpy as np

#string e data
import unidecode #limpeza
import re #regex

#tempo
import time
from time import sleep

#web scraping
import requests
from bs4 import BeautifulSoup

#automação
from selenium import webdriver #navegador
from selenium.webdriver.common.keys import Keys #comandos do teclado
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait #esperar
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyautogui #

import warnings #Warning inuteis
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

#manipulação do diretorio
import os

In [3]:
# FUNÇÕES XPATH
def xpath_element(xpath):
    return browser.find_element("xpath", xpath)

def xpath_click(xpath):
    try:
        xpath_element(xpath).click()
    except:
        sleep(1)
        xpath_element(xpath).click()
    
def xpath_send_keys(xpath, texto):
    xpath_element(xpath).send_keys(texto)
    
def xpath_clear(xpath):
    xpath_element(xpath).clear()

def existencia(caminho):
    elementos = browser.find_elements(By.XPATH, caminho)
    return bool(elementos)

In [4]:
#FUNÇÕES
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)

def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

In [5]:
#browser.quit(); sleep(5)
browser = webdriver.Firefox()

#fazendo login
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
xpath_send_keys('//*[@id="MainContent_LoginUser_UserName"]', 'gabriel_s_anjos@yahoo.com') #e-mail
xpath_send_keys('//*[@id="MainContent_LoginUser_Password"]', 'Gabriel2022!') #senha
xpath_click('//*[@id="MainContent_LoginUser_btnLogin"]') #login

In [6]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal')

In [8]:
#click setor/profissional
xpath_click('//*[@id="s2id_autogen1"]/a/span[1]')

#click [profissional
xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')

# Capturando setores:

In [9]:
#click criar novo plantão
def criar_plantao():
    xpath_click('//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button')

criar_plantao() #abrindo para capturar os setores

def esperar_elemento_aparecer(elemento, tempo):
    sleep(1)
    WebDriverWait(browser, tempo).until(visibility_of(xpath_element(elemento)))
esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
print('apareceu')

apareceu


In [10]:
#CAPTURANDO DADOS DO UNIDADE E SETOR
xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
setor_html = xpath_element('//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1

        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
print('OK')

OK


In [11]:
#CARREGANDO OS DADOS PARA SUBIR NO PEGA PLANTÃO
df = pd.read_excel('df_gabs.xlsx') # df_gabs # df_continuar
df = df.sort_values(['nome', 'data']).reset_index(drop=True)

df['setor'][(df['setor'] == 'FDS - C.CIRURGICO ROTINA') & (df['unidade'] == 'SANTA ISABEL')] = 'FDS - MANHA - C.CIRURGICO'

print(len(df))
df.sample(3)

414


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2
36,[28/06/2024 06:50:13] Augusto de Castro Santos...,28/06/2024,AUGUSTO DE CASTRO SANTOS QUEIROZ,28/06 MORUMBI - CASTRO ENTRADA 06:00,NaN,28/06/2024,MORUMBI,NaN,ENTRADA,06:00,07:00,01:00,6h,1,SEMANA - 6HS,semana,MORUMBI
71,[26/06/2024 18:35:51] Caroline Firmino Lemes d...,26/06/2024,CAROLINE FIRMINO LEMES DA SILVA,26/06 MORUMBI LEMES SAÍDA 17:40,NaN,26/06/2024,MORUMBI,NaN,SAIDA,13:00,18:00,05:00,tarde,3,SEMANA - TARDE - C.CIRURGICO,semana,MORUMBI
73,[27/06/2024 19:58:14] Caroline Firmino Lemes d...,27/06/2024,CAROLINE FIRMINO LEMES DA SILVA,27/06 JABAQUARA LEMES SAÍDA 19:55,NaN,27/06/2024,JABAQUARA,NaN,SAIDA,19:00,20:00,01:00,noturno saida,4,SEMANA - NOTURNO - SAIDA,semana,JABA


In [12]:
#TESTANDO COMPATIBILIDADE DOS DADOS CAPTURADOS E DADOS CARREGADOS
for X in range(0, len(df)):
    if len(scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][X]) & 
                                 (scrap_unidades.momento == df['setor'][X])]) < 1:
        print(X)
        print(df['unidade'][X], '-', df['setor'][X])
        #print('index', ind)
        #print(scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #print('codigos:', unidade, setorzin)
        

print("="*20, "TUDO OK", "="*20)

==================== TUDO OK ====================


# =========================================================

# FUNÇÕES

In [13]:
def subir_pg():
    browser.execute_script("window.scrollTo(0, 0);")

In [14]:
def reset_selecao():
    xpath_click('/html')#; sleep(0.3)
    #xpath_click('/html')


def fechar_caixa_de_selecao():
    # Localiza um elemento que é seguro para clicar (por exemplo, o corpo da página)
    elemento_seguro = browser.find_element(By.TAG_NAME, 'body')
    # Cria uma ação para clicar no elemento seguro
    ActionChains(browser).move_to_element(elemento_seguro).click().perform()
def reset_selecao1():
    fechar_caixa_de_selecao(); sleep(0.3)
    fechar_caixa_de_selecao()


def reset_selecao2():
    posicao_antiga = pyautogui.position() 
    
    #click no vazio
    pyautogui.click(x=66, y=158); sleep(0.3)
    pyautogui.click(x=66, y=158)
    
    pyautogui.moveTo(posicao_antiga) 
reset_selecao2()

In [15]:
#selecionando setor
def setor(unidade, setor):
    #print('base:', unidade, '-', setor)
    xpath_click('//*[@id="s2id_ddlServiceSector"]/a/span[1]')
    ind = scrap_unidades.loc[(scrap_unidades.unidade == unidade) & 
                             (scrap_unidades.momento == setor)].index.values[0]
    unidade_sel = scrap_unidades['unidade_count'][ind]
    setor_sel = scrap_unidades['plt_count'][ind]
    xpath_click('//*[@id="select2-drop"]/ul/li['+str(unidade_sel)+']/ul/li['+str(setor_sel)+']/div')   
setor("SANTA ISABEL", "SEMANA - TARDE - C.CIRURGICO")

#inserir e selecionar profissional
def nome(nome_profissional):
    #listar todos
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    xpath_click('//*[@id="service-ckb-list-all-owner"]')
    sleep(0.3)
    xpath_click('//*[@id="s2id_ddlProfessionalA"]/a/span[1]')
    xpath_send_keys('//*[@id="select2-drop"]/div/input', nome_profissional)
    #Dando enter
    xpath_send_keys('//*[@id="select2-drop"]/div/input', Keys.RETURN)
nome('OTTO MITTERMAYER')

#inserir horario e duração
def hora(inicio, duracao):
    #Inicio do plantão
    xpath_clear('//*[@id="txtServiceStartTime"]')
    xpath_send_keys('//*[@id="txtServiceStartTime"]', inicio)
    sleep(0.3)
    xpath_clear('//*[@id="txtServiceDuration"]')
    xpath_send_keys('//*[@id="txtServiceDuration"]', duracao)
hora('13:30', '05:00')

In [20]:
#inserir data
def data(data_selecionada):
    i_chato2 = str(5) #4 e 5
    i_chato = str(3) #2 e 3

    xpath_clear('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]')
    #xpath_clear('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]')
    sleep(0.3)
    
    xpath_send_keys('/html/body/div['+i_chato2+']/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div['+i_chato+']/input[1]', data_selecionada)
    #xpath_send_keys('/html/body/div[4]/div/div[2]/div[2]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[3]/input[2]', data_selecionada)
    reset_selecao2()
data("25/05/2024")

In [23]:
#inserindo pontuação
def pontuacao(pontos):
    xpath_clear('//*[@id="txtServiceValue"]')
    xpath_send_keys('//*[@id="txtServiceValue"]', pontos) #Pontos

pontuacao("666")
'''
//*[@id="select2-drop"]/ul/li[3]/div
1 - Normal
2 - DESLOCAMENTO
3 - VOLUNTÁRIO
4 - COBERTURA
'''

#selecionar tipo de plantão e ajustando tempo
def tipo_do_registro(duracao, sem_ou_fds, tipo_registro):
    if tipo_registro == 'extra':
        print('EXTRA')
        sleep(1)
        reset_selecao2()
        
        xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        #browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        xpath_click('//*[@id="select2-drop"]/ul/li[3]/div')
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(200)
            pontuacao(200)
            print('duração >= 5 => 200 pontos')
        else:
            pass
    elif tipo_registro == 'DESLOCADO':
        print('DESLOCADO')
        sleep(1)
        reset_selecao2()
        
        xpath_click('//*[@id="s2id_add-service-service-types"]/a/span[1]')
        #browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        xpath_click('//*[@id="select2-drop"]/ul/li[2]/div')
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)
    else:
        if int(duracao[:2]) >= 5 and sem_ou_fds == "semana":
            pontuacao(100)
            pontuacao(100)
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(1)

#tipo_do_registro("05:00", "semana", "extra")
#tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])
        
#salvando plantão
def salvar_plantao():
    xpath_click('//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]')
    
#salvar_plantao()

#fechando plantão
def fechar():
    xpath_click('//*[@id="modal-add-service"]/div/div[1]/button')

fechar()

# =========================================================

# RODANDO NO PEGA

In [24]:
len(df)

414

In [25]:
df['controle'].value_counts()

controle
DESLOCADO       10
DESLOCADO 6H     9
Name: count, dtype: int64

In [26]:
H=0

In [35]:
count = 0
for h in range(H, len(df)):
    #####################################################################################################################
    if h > 0:
        if df['nome'][h] != df['nome'][h-1] or count == 0:
            print('='*20, df['nome'][h], '='*20)
            count += 1
    #####################################################################################################################
    #try:
    print('[',h,'] |','dia:', df['data'][h],'| mensagem:', df['mensagem'][h])
    
    #click criar novo plantão
    criar_plantao()
    
    #SETOR - UNIDADE
    #esta disponivel o botão de setor ?
    esperar_elemento_aparecer('//*[@id="s2id_ddlServiceSector"]/a/span[1]', 10)
    #Selecionando Setor
    setor(df['unidade'][h], df['setor'][h])
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    esperar_elemento_aparecer('//*[@id="service-ckb-list-all-owner"]', 10)
    nome(df['nome'][h])
    sleep(1)

    #DATA DO PLANTÃO
    data(df['data'][h])
    
    #INICIO DO PLANTÃO -- DURAÇÃO DO PLANTÃO
    hora(df['ENTRADA'][h], df['DURAÇÃO'][h])
    sleep(1)
        
    #TIPO
    tipo_do_registro(df['DURAÇÃO'][h], df['sem_ou_fds'][h], df['extra'][h])
    
    #SALVAR PLANTÃO
    sleep(1)
    salvar_plantao()
    salvar_plantao()
    salvar_plantao()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    chatao = 5 #6 5 4
    if xpath_element('/html/body/div['+str(chatao)+']/div/div[2]/div[1]').text != "":
        if df['PERIODO'][h] == '6h':
            ##############################################################################
            #xpath_click('//*[@id="ckb-add-service-ignore-conflict"]')#estou ciente
            #sleep(1)
            #salvar_plantao()
            #salvar_plantao()
            #salvar_plantao()
            #df.loc[int(h), 'controle'] = "NORM DUPLI"
            #print("NORM DUPLI")

            ##############################################################################
            fechar()
            sleep(0.3)
            #if existencia('//*[@id="modal-add-service"]/div/div[1]/button'):
                #fechar()
            df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
        else:
            fechar()
            df.loc[int(h), 'controle'] = "PROBS"; print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"; print("NORM")
    #####################################################################################################################
    '''except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")
        if existencia('//*[@id="modal-add-service"]/div/div[1]/button'):
            fechar()
            print('*'*50)
            df.loc[int(h), 'controle'] = "ERRO_REPETIR"; print("ERRO_REPETIR")
            print('*'*50)
        else:
            print('-VERIFICAR-'*10)
            break'''
            
    #####################################################################################################################
    # praticamente um # finally
    WebDriverWait(browser, 10).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    #####################################################################################################################
    if H % 25 == 0:
        print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs.xlsx", index=False)
    if df['nome'][h] == df['nome'][h+1]:
        print('='*10)
        
print("="*20, 'TERMINOU', "="*20)
print('Total  ', len(df))
print('='*80, '\n', df['controle'].value_counts(), '\n', '='*80); df.to_excel("df_gabs.xlsx", index=False)

==================== TERMINOU ====================
Total   414
 controle
PROBS    318
NORM      96
Name: count, dtype: int64 


In [29]:
chatao = 5
xpath_element('/html/body/div['+str(chatao)+']/div/div[2]/div[1]').text

'Informações:\nEsta ação irá gerar o(s) seguinte(s) alerta(s):\nPlantões no mesmo horário (1)\nEstou ciente e quero salvar mesmo assim\n×'

In [ ]:
df.loc[int(h), 'controle'] = "NORM"; print("NORM")
H += 1

NORM


In [33]:
df['nome'][df['nome'] == '~ ICARO MOURA'] = 'icaro de MOURA SOUSA'.upper()

# Testes:

# ============================================================

# Salvando restante:

In [36]:
print(df['controle'].value_counts())
print(df['controle'].value_counts(normalize=True)*100)

controle
PROBS    318
NORM      96
Name: count, dtype: int64
controle
PROBS    76.811594
NORM     23.188406
Name: proportion, dtype: float64


In [37]:
# Registros com Problemas Corrigiveis ou Não
df_gabs = df[df['controle'] != 'NORM']
df_gabs.reset_index(drop=True, inplace=True)
print("probs:", len(df_gabs))

# Registros Perfeitos ja feitos
df = df[df['controle'] == 'NORM']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df))

os.remove("df_gabs.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df)
print('.')
salvando_nome_arquivo('df_gabs2.xlsx', df_gabs)

probs: 318
ok: 96
2818  +=  96 => 2913
.
318


# =======================================================================

In [38]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [39]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

# =======================================================================

In [40]:
######### ######### FUNÇÃO DE CAPTURA ######### ######### 
#capturando registros
unidades = ['ADMINISTRACAO','ALPHA','ANALIA','AVICCENA',
            'ITAIM','JABAQUARA','MATERNIDADE','MORUMBI',
            'OSASCO','VILLA','VNS','SANTA ISABEL']

def funcao():
    lista_tabela = []
    for i in range(1, 6): #passando por cada semana[1 a 5]
    ##print('semana:', i, "=="*20)
        for t in range(1, 8): #passando por cada dia[1 a 7] dentro da semana
            reg = xpath_element('/html/body/form/main/div[2]/div[2]/table/tbody/tr['+str(i)+']/td['+str(t)+']').text
            content = reg.splitlines() #deixando as palavras em um item dentro de uma lista
            content = [unidecode.unidecode(i.strip().upper()) for i in content] #tratando cada palavra

            dia = content[0] #dia do registro
            del(content[0]) #deletando o dia, ja que foi coletado
            del(content[-1]) #deletando residuo do wepscraping

            if len(content) != 0: #ignorando dias em branco (sem registro) ¹
                #print('dia:', t)

                ##print('dia:', dia)
                #print(content)

                count_hora = 0 #count para capturar a hora de entrada e saida, count=1(entrada) | count=2(saida)
                lista_control = []
                for y in content: #passando um for sobre cada item da lista
                    if y in unidades: # capturando a unidade e dando inicio a novo registro novo
                        #print('================unidade:', y)
                        unidade = y
                    elif y[2] == ':': #reconhecendo se o item é hora (HH*:*mm)
                        count_hora += 1
                        if count_hora == 1: #capturando entrada
                            #print('hora entrada:', y)
                            lista_control.append(y)
                        elif count_hora == 2: #capturando saida
                            #print('hora saida:', y)
                            lista_control.append(y)
                            ##print('lista teste', lista_control)
                            lista_tabela.append(lista_control)
                            lista_control = []
                            count_hora = 0
                    else: #capturando dia, unidade, e periodo
                        #print('.')
                        #print('periodo:', y)
                        lista_control.append(dia)
                        lista_control.append(unidade)
                        lista_control.append(y)
                        lista_control.append(i)
                        lista_control.append(t) #dia semanal

                ##print('='*20)    

            else:
                continue #ignorando dias em branco (sem registro) ²
    
    tabela = pd.DataFrame(lista_tabela, columns = ['Dia','Unidade','Periodo','Semana','Dia Semana','Entrada','Saida'])
    
    #limpeza da tabela
    #########3
    tabela['Dia'] = tabela['Dia'].apply(lambda x: int(x))

    #recolhendo os dias do mes passado que ainda aparecem na primeira semana
    dias_mes_passado = list(tabela[(tabela['Dia'] > 20) & (tabela['Semana'] == 1)].index) 
    tabela.drop(dias_mes_passado, axis=0, inplace=True) #excluindo
    tabela.reset_index(drop=True, inplace=True) #resetando index

    #Inserindo a ordem dos registros
    tabela['ordem'] = ''

    for i in tabela['Dia'].unique():
        for t in range(0, len(tabela[tabela['Dia'] == i].index)):
            tabela['ordem'][tabela[tabela['Dia'] == i].index[t]] = t+1

    #Inserindo coluna de data        
    tabela['data'] = tabela['Dia'].apply(lambda x: str(x))

    for i in range(len(tabela)):
        if len(tabela['data'][i]) == 1:
            tabela['data'][i] = "0" + tabela['data'][i]

        tabela['data'][i] = tabela['data'][i] + "/" + mes + "/2024" 

    tabela = tabela.query("Unidade != 'ADMINISTRACAO'")
    tabela = tabela[~((tabela['Semana'] >= 5) & (tabela['Dia'] < 10))]
    tabela.reset_index(drop=True, inplace=True) #resetando index
    
    tabela['PERIODO'] = np.nan
    for i in range(0, len(tabela)):
        if int(tabela['Entrada'][i][:2]) <= 6:
            tabela['PERIODO'][i] = '6h'
        elif int(tabela['Entrada'][i][:2]) >= 7 and int(tabela['Entrada'][i][:2]) <= 12:
            tabela['PERIODO'][i] = 'manha'
        elif int(tabela['Entrada'][i][:2]) >= 13 and int(tabela['Entrada'][i][:2]) <= 18:
            tabela['PERIODO'][i] = 'tarde'
        else:
            tabela['PERIODO'][i] = 'noturno saida'

    tabela['periodo'] = np.nan
    tabela['periodo'][tabela['PERIODO'] == '6h'] = 1
    tabela['periodo'][tabela['PERIODO'] == 'manha'] = 2
    tabela['periodo'][tabela['PERIODO'] == 'tarde'] = 3
    tabela['periodo'][tabela['PERIODO'] == 'noturno saida'] = 4
    
    return tabela
    
tabela = funcao()
tabela

,Dia,Unidade,Periodo,Semana,Dia Semana,Entrada,Saida,ordem,data,PERIODO,periodo


# =======================

# DIMINUINDO TEMPOS & DESLOCADO

In [41]:
#df_backup = df_gabs
df_backup = pd.read_excel('df_gabs2.xlsx')

print(len(df_backup))
df_backup['setor_inicio'] = df_backup['setor'].apply(lambda x: x.replace(' - C.CIRURGICO',''))

df_backup = df_backup.sort_values(['nome','data','periodo']).reset_index(drop=True)
df_backup.reset_index(drop=True, inplace=True)

df = df_backup.copy()

df.head()

318


,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds,unidade2,setor_inicio
0,[26/06/2024 10:20:48] Alessandre Luiz Braga Ju...,26/06/2024,ALESSANDRE LUIZ BRAGA JUNIOR,26/06 ANALIA - ALESSANDRE ENTRADA 06:00,PROBS,26/06/2024,ANALIA,NaN,ENTRADA,06:00,07:00,01:00,6h,1,SEMANA - 6HS,semana,ANALIA,SEMANA - 6HS
1,[26/06/2024 10:21:12] Alessandre Luiz Braga Ju...,26/06/2024,ALESSANDRE LUIZ BRAGA JUNIOR,26/06 ANALIA - ALESSANDRE SAIDA 10:20,PROBS,26/06/2024,ANALIA,NaN,SAIDA,07:00,10:30,03:30,manha,2,SEMANA - MANHA - C.CIRURGICO,semana,ANALIA,SEMANA - MANHA
2,[26/06/2024 06:46:29] Alexandre Amaral Lucas: ...,26/06/2024,ALEXANDRE AMARAL LUCAS,26/06 OSASCO - ALEXANDRE AMARAL ENTRADA 06:00,PROBS,26/06/2024,OSASCO,NaN,ENTRADA,06:00,07:00,01:00,6h,1,SEMANA - 6HS,semana,OSASCO,SEMANA - 6HS
3,[26/06/2024 22:40:49] Alexandre Amaral Lucas: ...,26/06/2024,ALEXANDRE AMARAL LUCAS,26/06 OSASCO - ALEXANDRE AMARAL SAIDA 22:40,PROBS,26/06/2024,OSASCO,NaN,SAIDA,19:00,23:00,04:00,noturno saida,4,SEMANA - NOTURNO - SAIDA,semana,OSASCO,SEMANA - NOTURNO - SAIDA
4,[28/06/2024 07:22:09] Alexandre Amaral Lucas: ...,28/06/2024,ALEXANDRE AMARAL LUCAS,28/06 OSASCO ALEXANDRE AMARAL ENTRADA 07:14 (...,PROBS,28/06/2024,OSASCO,NaN,ENTRADA,07:00,13:00,06:00,manha,2,SEMANA - MANHA - C.CIRURGICO,semana,OSASCO,SEMANA - MANHA


In [42]:
from selenium.webdriver.common.action_chains import ActionChains

action_chains = ActionChains(browser)

In [43]:
len(df_backup)

318

In [44]:
df_backup['controle'].value_counts()

controle
PROBS    318
Name: count, dtype: int64

In [47]:
def atualizando_plantao():
    xpath_click('/html/body/div[5]/div/div[1]/div[2]/button[5]')
    #xpath_click('/html/body/div[4]/div/div[1]/div[2]/button[5]')
atualizando_plantao()

In [48]:
count = 0

In [49]:
count_freq(df_backup, 'controle')

,Freq,Perc
controle,,
PROBS,318,100.0


In [50]:
def xpath_estranho(i):
    xpaths = [
        '/html/body/div[12]/div/input',
        '/html/body/div[11]/div/input',
        '/html/body/div[10]/div/input',
        '/html/body/div[9]/div/input',
        '/html/body/div[8]/div/input',
        '/html/body/div[7]/div/input',
        '/html/body/div[6]/div/input',
    ]
    
    for xpath in xpaths:
        try:
            xpath_send_keys(xpath, i)
            return  # Sai da função se encontrar um xpath que funcione
        except:
            continue  # Tenta o próximo xpath se ocorrer uma exceção
    
    print('erro')  # Imprime 'erro' se nenhum xpath funcionar

xpath_estranho(i)

erro


In [55]:
############################-CODIGO-############################
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome = df_nome.reset_index(drop=True)

    subir_pg(); sleep(0.5)
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')
    
    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))

    tabela = funcao(); sleep(0.3)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        print(df_nome['mensagem'][t])
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]] # & (tabela['Periodo'] == df_nome["setor"][t])]
        
        if df_nome['PERIODO'][t] == 'manha' and df_nome['sem_ou_fds'][t] == 'fds':
            dia_sel = dia_sel[dia_sel['Entrada'] == '07:00']
        else:
            dia_sel = dia_sel[dia_sel['Periodo'].str.startswith(df_nome["setor_inicio"][t])]
        
        dia_sel.reset_index(drop=True, inplace=True)
        #dia_sel
        
        if len(dia_sel) == 0:
            df_backup['controle'][count] = "PROB"; print(count, "-> PROB")
        elif len(dia_sel) == 2:
            #abrindo/selecionando periodo conforme localização
            xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
            xpath_click(xpath_localizacao)
            #Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(0.3)
            apagando_registro(); sleep(0.3)
            df_backup['controle'][count] = "OK"; print(count, "-> OK")
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
            sleep(1)
        else:
            #abrindo/selecionando periodo conforme localização
            xpath_localizacao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][0])+']/li'
            xpath_click(xpath_localizacao)
            #Esperar componentes carregarem
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
            sleep(1)
            hora(df_nome['ENTRADA'][t], df_nome['DURAÇÃO'][t])
            if dia_sel['Unidade'][0] != df_nome['unidade'][t]:
                setor(df_nome['unidade'][t], df_nome['setor'][t])
            else:
                pass
            
            tipo_do_registro(df_nome['DURAÇÃO'][t], df_nome['sem_ou_fds'][t], df_nome['extra'][t])
            sleep(0.3)
            reset_selecao2()
            #salvar plantão
            sleep(0.3)
            atualizando_plantao(); atualizando_plantao(); sleep(0.3)
            df_backup['controle'][count] = "OK"; print(count, "-> OK")
            WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
            sleep(1)
            
        count += 1 
        
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n'); df_backup.to_excel("df_gabs2.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
            
print("="*50, 'TERMINOU', "="*50)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_gabs2.xlsx", index=False)

==================== THIAGO SILVA PINTO ====================
27/06 SANTA  -  T. SILVA  -  ENTRADA 06:00
303 -> OK
==================== TURIBIO RODRIGUES FREIRE NETO ====================
26/06 MORUMBI  -  TURIBIO ENTRADA: 06:15
304 -> OK
26/06 MORUMBI  -  TURIBIO SAÍDA: 12:00
duração >5 -> 100 pontos
305 -> OK
==================== VANESSA PACIULLO MAROSSI ====================
26/06 VILLA  -  VANESSA SAÍDA 14:45
306 -> OK
==================== VICTOR AUGUSTO DE SOUZA MORAES ====================
26/06 OSASCO  -  V.AUGUSTO SAÍDA 23:00
307 -> OK
27/06 ANÁLIA  -  V.AUGUSTO SAÍDA 16:25
308 -> OK
==================== VITOR MACIEL DE SOUSA PINTO ====================
25/06 ANÁLIA  -  MACIEL SAÍDA 19:50
309 -> OK
26/06 ANÁLIA  -  MACIEL SAÍDA 19:55
310 -> OK
==================== VIVIAN CRISTINE DOS ANJOS E SILVA ====================
26/06 ANALIA VIVIAN ENTRADA 13:00
duração >5 -> 100 pontos
311 -> OK
26/06 ANALIA VIVIAN SAIDA 20:40
312 -> OK
==================== VIVIANNA BOVERIO GALZERANI ========

In [ ]:
count += 1 

In [ ]:
(31/54)*100

In [ ]:
df_backup['nome'][df_backup['nome'] == 'MARCO VINICIUS COSTA CARVALHO'] = 'MARCOS VINICIUS COSTA CARVALHO'

In [ ]:
df_backup.loc[int(count), 'controle'] = "OK"
count += 1
print(df_backup['nome'][count])
print(count)

In [56]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

###############################################################

salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
os.remove("df_gabs2.xlsx")
print('.')
salvando_nome_arquivo('df_giov2.xlsx', df_prob)

probs ok: 316
probs: 2
2913  +=  316 => 2980
.
2


# =======================

# ==============================================================

# DISPENSADOS

In [57]:
df_backup = pd.read_excel('df_dispensados.xlsx')
print(len(df_backup))
df_backup = df_backup.drop_duplicates(subset=['chat'])
df_backup = df_backup.sort_values(['nome', 'dia']).reset_index(drop=True)
#df_backup = df_backup.sort_values(['nome', 'data', 'periodo'], ascending=[True, True, False]).reset_index(drop=True)
print(len(df_backup))
#df_backup.sample(3)

12
12


In [58]:
count = 0

In [59]:
df_backup['controle'] = np.nan

In [60]:
df_backup["nome"] = df_backup["nome"].apply(lambda x: x.replace("  ", " "))

In [64]:
def apagando_registro():
    chatao = str(5) #4 ou 5
    #Apagando
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[2]/button[1]').click()
    #Clicando em continuar
    sleep(0.5)
    browser.find_element("xpath", '/html/body/div['+chatao+']/div/div[1]/div[3]/button[2]').click()
apagando_registro()

In [66]:
#Para cada profissional na lista
df = df_backup[count:]

for i in df['nome'].unique():
    print('='*20, i, '='*20)
    df_nome = df[df['nome'] == i]
    df_nome.reset_index(drop=True, inplace=True)
    
    xpath_click('//*[@id="s2id_ddlProfessionals"]/a/span[1]'); sleep(1)
    xpath_estranho(i)
    xpath_click('//*[@id="select2-drop"]/ul/li/div')

    #esperando carregamento do site
    WebDriverWait(browser, 30).until_not(visibility_of(xpath_element('//*[@id="loader"]')))

    tabela = funcao()    
    sleep(2)
    
    #Para cada registro na lista_nome
    for t in range(len(df_nome)):
        print(df_nome['mensagem'][t])
        dia_sel = tabela[tabela['data'] == df_nome['data'][t]]
        dia_sel.reset_index(drop=True, inplace=True)
        
        if len(dia_sel) == 0:
            print(count, "-> Ja apagado")
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"
        else:
            for r in range(len(dia_sel)-1, -1, -1):
                print('PRINCIPAL')
                #abrindo/selecionando periodo conforme localização
                dia_ordem = r
                xpath_textao = '//*[@id="shift-monthly"]/div[2]/table/tbody/tr['+str(dia_sel['Semana'][0])+']/td['+str(dia_sel['Dia Semana'][0])+']/ul['+str(dia_sel['ordem'][dia_ordem])+']/li'
                xpath_click(xpath_textao)
                #Esperar componentes carregarem
                WebDriverWait(browser, 10).until(visibility_of(xpath_element('//*[@id="service-ckb-list-all-owner"]')))
                sleep(2)
                #Apagando registro
                apagando_registro()
                #esperando carregamento do site para prosseguir
                WebDriverWait(browser, 20).until_not(visibility_of(xpath_element('//*[@id="modal-add-service"]/div/div[1]/button')))
            
            tabela = funcao() #atualizando tabela de registros
            sleep(2)
            print(count, '-> OK')
            print(df_nome['chat'][t] + " - " + df_nome['data'][t])
            df_backup['controle'][count] = "OK"
            
        count += 1 
        if count % 25 == 0:
            print('='*80, '\n', df_backup['controle'].value_counts(), '\n', '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)
        else:
            if t != len(df_nome)-1:
                print('='*20)
    
print('='*25, 'TERMINOU', '='*25)
print('Total  ', len(df_backup))
print('='*80, df_backup['controle'].value_counts(), '='*80); df_backup.to_excel("df_dispensados.xlsx", index=False)

==================== CARLOS AUGUSTO TREVISAN ====================
26/06 ITAIM  -  CARLOS DISPENSADO
2 -> Ja apagado
[26/06/2024 08:58:10] Carlos Augusto Trevisan: 26/06 ITAIM - Carlos dispensado - 26/06/2024
==================== DANIELLEN LINS LOURENCO ====================
26/06 MORUMBI  -  DANIELLEN DISPENSADO
3 -> Ja apagado
[26/06/2024 16:44:21] Daniellen Lins Lourenço: 26/06 MORUMBI - Daniellen dispensada - 26/06/2024
==================== FELIPE COELHO DE SOUZA ====================
26/06 ITAIM  -  COELHO DISPENSADO
4 -> Ja apagado
[26/06/2024 10:39:02] Felipe Coelho de Souza: 26/06 Itaim - Coelho dispensado - 26/06/2024
==================== GUILHERME SILVEIRA LOPES ====================
28/06 MORUMBI  -  SILVEIRA DISPENSADO.
PRINCIPAL
PRINCIPAL
5 -> OK
[28/06/2024 08:38:40] Guilherme Silveira Lopes: 28/06 Morumbi - Silveira dispensado. - 28/06/2024
==================== KAREN MANOELA ROSA ====================
26/06 KROSA ITAIM DISPENSADO
6 -> Ja apagado
[26/06/2024 09:32:01] Karen Ma

In [65]:
df_backup['controle'][count] = "OK"
count += 1

In [ ]:
df_backup['nome'][df_backup['nome'] == "ICARO MOURA DE SOUSA"] = "ICARO DE MOURA SOUSA"

In [67]:
df_corrigidos = df_backup[df_backup['controle'] == "OK"]
print("probs ok:", len(df_corrigidos))

df_prob = df_backup[df_backup['controle'] != "OK"]
print("probs:", len(df_prob))

################################################################

os.remove("df_dispensados.xlsx")
salvando_nome_arquivo('Feitos.xlsx', df_corrigidos)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

probs ok: 12
probs: 0
2980  +=  12 => 2981
.
18  +=  0 => 18


In [68]:
browser.quit()

# MENSAGEM GIOVANELLI

In [69]:
df_feito = pd.read_excel("Feitos.xlsx")
print(len(df_feito))

df_prob0 = pd.read_excel("df_giov.xlsx")
print(len(df_prob0))
df_prob01 = pd.read_excel("df_giov2.xlsx")
print(len(df_prob01))

os.remove("df_giov.xlsx")
os.remove("df_giov2.xlsx")

df_prob = pd.concat([df_prob0, df_prob01])
df_prob = df_prob.drop_duplicates(subset=['chat'])
salvando_nome_arquivo('df_prob.xlsx', df_prob)

print(len(df_prob))

2981
18
2
162  +=  20 => 170
20


In [70]:
total = len(df_feito) + len(df_prob)

percent = len(df_feito) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros App:", len(df_feito), percent)

percent = len(df_prob) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros Giovanelli", len(df_prob), percent)
print("#"*30, "\n")

for nome in sorted(list(df_prob['nome'].unique())):
    print(nome, "="*10)
    selecao = df_prob.loc[df_prob['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")

Registros App: 2981 (99.33%)
Registros Giovanelli 20 (0.67%)
############################## 

ANA GABRIELA GEWEHR VALE SALMEN ==========
‎ANA GABRIELA GEWEHR VALE SALMEN SAIU


BRUNO STORNIOLI TAGUCHI LEMOS ==========
27/06 HIAE BRUNO LEMOS ENTRADA 13:00
27/06 HIAE SAIDA 17:20


CLAUDIO MUELLER KAKUDA ==========
27/06 MATERNIDADE  -  KAKUDA ENTADA 6:00


FABIO ANTONIO DE SOUZA OLIVEIRA ==========
28/06 ITAIM FABIO ENTRADA 6HS


MARCELO DE OLIVEIRA PERES ==========
28/06 ITAIM  -  MARCELO ENTRADA 13:00  (  DISTÂNCIA ) 


NATAN SOARES SILVEIRA ==========
25/06 HCT  -  NATAN  -  SAÍDA 19:15
25/06 HCT  -  NATAN  -  ENTRADA 05:30


PEDRO HENRIQUE DINIZ BOTELHO ==========
26/06 OSASCO  -  DINIZ ENTRADA 12:00


RAFAEL TADEU MARQUES ==========
27/06 HIAE RAFAEL TADEU SAÍDA 20:10


RENATA TOGEIRO BASTOS GODOY DE FARIA ==========
27/06 MORUMBI  -  RENATA TOGEIRO ENTRADA 5:00


RODRIGO ANTONIO REY GONZALEZ ==========
24/06 EXTERNA  -  REY ENTRADA 07:00
24/06 EXTERNA  -  REY SAÍDA 20:00


SABRINA 

In [ ]:
for i in df_prob['chat']:
    print(i)

In [ ]:
os.remove("df_prob.xlsx")
df_prob.to_excel("df_prob_enviado_2904.xlsx", index=False)